In [66]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
import string
import random
from nltk.tokenize import WordPunctTokenizer
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import LinearSVC

[nltk_data] Downloading package punkt to /home/shivansh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shivansh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('./rawdata.csv')
df.head()

,SENTIMENT,ID,DATE,BAKWAS,USER,TEXT,Unnamed: 6
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",NaN
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,NaN
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,NaN
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,NaN
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",NaN


In [32]:
zero = df[df.SENTIMENT == 0]
zero.count()

SENTIMENT     800000
 ID           800000
 DATE         800000
 BAKWAS       800000
 USER         800000
 TEXT         800000
Unnamed: 6         0
dtype: int64

In [34]:
four = df[df.SENTIMENT == 4]
four.count()

SENTIMENT     800000
 ID           800000
 DATE         800000
 BAKWAS       800000
 USER         800000
 TEXT         800000
Unnamed: 6         0
dtype: int64

In [51]:
zero = zero.head(10000)
four = four.head(10000)
dataset = pd.concat([zero, four])
dataset.to_csv('betterdata.csv', index=False)

In [65]:
def extract_csv():
    my_filtered_csv = pd.read_csv('./betterdata.csv', usecols=['SENTIMENT', 'TEXT'])
    return my_filtered_csv

def tokenize_tweets(my_csv):
    tweets = my_csv.TEXT.tolist()
    sentiments = my_csv.SENTIMENT.tolist()
    tokenizer = WordPunctTokenizer() 
    cleaned = []
    for i in range(0, len(tweets)):
        text = tweets[i]
        text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', text)
        text = re.sub("(@[A-Za-z0-9_]+)","", text)
        text = tokenizer.tokenize(text)
        element = [text, sentiments[i]]
        cleaned.append(element)
    return cleaned

def lemmatize_sentence(tweet_tokens, stop_words = ()):
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('V'):
            pos = 'v'
        else:
            pos = 'a'
        token = lemmatizer.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def create_lemmatized_sent(words):
    cleaned = []
    stop_words = stopwords.words('english')
    for i in range(0, len(words)):
        sent = lemmatize_sentence(words[i][0], stop_words)
        if len(sent) > 0:
            element = [sent, words[i][1]]
            cleaned.append(element)
    return cleaned

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def write_sent(sent):
    cleaned = []
    for i in sent:
        s = ""
        for j in i[0]:
            j = str(j)
            j = j + " "
            s = s + j
        s = remove_emoji(s)
        element = [s, i[1]]
        cleaned.append(element)
    df = pd.DataFrame(cleaned)
    df.to_csv('cleaned_data.csv', index=False)

my_csv = extract_csv()
words = tokenize_tweets(my_csv)
sent = create_lemmatized_sent(words)
write_sent(sent)

In [ ]:
df = pd.read_csv('cleaned_data.csv')
tweets = []
train_txt = []
test_txt = []
test_sent = []
train_sent = []

txt = df.text.tolist()
sent = df.sentiment.tolist()